<a href="https://colab.research.google.com/github/asadbek002/Camera_Calibaration/blob/master/huggyface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch


In [ ]:
# Загрузка датасета MRPC из коллекции GLUE
dataset = load_dataset('glue', 'mrpc')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# Инициализация токенизатора DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Инициализация модели DistilBERT для задачи классификации последовательностей
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=True)

# Токенизация данных
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# Выборка для тренировки и тестирования
train_dataset = tokenized_datasets['train'].shuffle(seed=42)
eval_dataset = tokenized_datasets['validation'].shuffle(seed=42)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    evaluation_strategy="epoch",     # стратегия оценки
    per_device_train_batch_size=8,   # размер батча для тренировки
    per_device_eval_batch_size=8,    # размер батча для оценки
    num_train_epochs=3,              # количество эпох
    weight_decay=0.01,               # коэффициент регуляризации
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
!pip install transformers[torch] accelerate -U


In [ ]:
import accelerate
import transformers

print(f"Accelerate version: {accelerate.__version__}")
print(f"Transformers version: {transformers.__version__}")


Accelerate version: 0.30.1
Transformers version: 4.41.1


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Загрузка датасета MRPC из коллекции GLUE
dataset = load_dataset('glue', 'mrpc')

# Инициализация токенизатора DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Инициализация модели DistilBERT для задачи классификации последовательностей
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Формат данных для тренировки
tokenized_datasets = tokenized_datasets.remove_columns(['idx', 'sentence1', 'sentence2'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')

# Разделение данных
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

# Настройка аргументов тренировки
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    eval_strategy="epoch",           # стратегия оценки
    per_device_train_batch_size=8,   # размер батча для тренировки
    per_device_eval_batch_size=8,    # размер батча для оценки
    num_train_epochs=3,              # количество эпох
    weight_decay=0.01,               # коэффициент регуляризации
)

# Инициализация тренера и тренировка модели
trainer = Trainer(
    model=model,                       # модель для тренировки
    args=training_args,                # параметры тренировки
    train_dataset=train_dataset,       # тренировочные данные
    eval_dataset=eval_dataset,         # данные для оценки
    tokenizer=tokenizer                # токенизатор
)

# Запуск тренировки
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U


In [ ]:
import accelerate
import transformers

print(f"Accelerate version: {accelerate.__version__}")
print(f"Transformers version: {transformers.__version__}")


Accelerate version: 0.30.1
Transformers version: 4.41.1


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Загрузка датасета MRPC из коллекции GLUE
dataset = load_dataset('glue', 'mrpc')

# Инициализация токенизатора DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Инициализация модели DistilBERT для задачи классификации последовательностей
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Формат данных для тренировки
tokenized_datasets = tokenized_datasets.remove_columns(['idx', 'sentence1', 'sentence2'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')

# Разделение данных
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

# Настройка аргументов тренировки
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    eval_strategy="epoch",           # стратегия оценки
    per_device_train_batch_size=8,   # размер батча для тренировки
    per_device_eval_batch_size=8,    # размер батча для оценки
    num_train_epochs=3,              # количество эпох
    weight_decay=0.01,               # коэффициент регуляризации
)

# Инициализация тренера и тренировка модели
trainer = Trainer(
    model=model,                       # модель для тренировки
    args=training_args,                # параметры тренировки
    train_dataset=train_dataset,       # тренировочные данные
    eval_dataset=eval_dataset,         # данные для оценки
    tokenizer=tokenizer                # токенизатор
)

# Запуск тренировки
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
!pip uninstall -y accelerate transformers
!pip install transformers[torch] accelerate -U


Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Successfully uninstalled accelerate-0.30.1
Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 14.5 MB/s eta 0:00:00
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)


In [ ]:
_from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Загрузка датасета MRPC из коллекции GLUE
dataset = load_dataset('glue', 'mrpc')

# Инициализация токенизатора DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Инициализация модели DistilBERT для задачи классификации последовательностей
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding='max_length')

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Формат данных для тренировки
tokenized_datasets = tokenized_datasets.remove_columns(['idx', 'sentence1', 'sentence2'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format('torch')

# Разделение данных
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']

# Настройка аргументов тренировки
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    eval_strategy="epoch",           # стратегия оценки
    per_device_train_batch_size=8,   # размер батча для тренировки
    per_device_eval_batch_size=8,    # размер батча для оценки
    num_train_epochs=3,              # количество эпох
    weight_decay=0.01,               # коэффициент регуляризации
)

# Инициализация тренера и тренировка модели
trainer = Trainer(
    model=model,                       # модель для тренировки
    args=training_args,                # параметры тренировки
    train_dataset=train_dataset,       # тренировочные данные
    eval_dataset=eval_dataset,         # данные для оценки
    tokenizer=tokenizer                # токенизатор
)

# Запуск тренировки
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.390224


Epoch,Training Loss,Validation Loss
1,No log,0.390224


KeyboardInterrupt: 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [10]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="Asadbek1/my_model1", private=False)


RepoUrl('https://huggingface.co/Asadbek1/my_model1', endpoint='https://huggingface.co', repo_type='model', repo_id='Asadbek1/my_model1')

In [11]:
model.save_pretrained("my_model1")
tokenizer.save_pretrained("my_model1")

api.upload_folder(
    folder_path="my_model1",
    repo_id="Asadbek1/my_model1",
    repo_type="model"
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Asadbek1/my_model1/commit/c8b5ca05fb1214928b933fd50e61baabe0359a54', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c8b5ca05fb1214928b933fd50e61baabe0359a54', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
!git init
!git add .
!git commit -m "Initial commit"

!git remote add origin https://github.com/asadbek002/NLP2023.git
!git branch -M main
!git push -u origin main



hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ae8ab3800571.(none)')
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/asadbek002/NLP2023.git'


In [14]:
!git config --global user.email "asad1bek@mail.ru
!git config --global user.name "ABDULKHAKOV ASADBEK"


/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file
